In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


<b>Loading dependencies</b>

In [2]:
import pandas as pd
import time
df_fkrt=pd.read_csv('/content/drive/My Drive/flipkart_com-ecommerce_sample.csv',encoding="ISO-8859-1")
df_amzn=pd.read_csv('/content/drive/My Drive/amz_com-ecommerce_sample.csv',encoding="ISO-8859-1")

In [3]:
len(df_fkrt)

20000

<b>Data Preprocessing</b>

In [4]:
df_fkrt.isnull().sum()

uniq_id                       0
crawl_timestamp               0
product_url                   0
product_name                  0
product_category_tree         0
pid                           0
retail_price                 78
discounted_price             78
image                         3
is_FK_Advantage_product       0
description                   2
product_rating                0
overall_rating                0
brand                      5864
product_specifications       14
dtype: int64

<b>Removing null values</b>

In [5]:
df_fkrt=df_fkrt.dropna(subset=["retail_price","discounted_price","image","description","product_specifications","brand"])
df_amzn=df_amzn.dropna(subset=["retail_price","discounted_price","image","description","product_specifications","brand"])
df_fkrt.reset_index(drop=True,inplace=True)
df_amzn.reset_index(drop=True,inplace=True)

In [6]:
df_fkrt.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [7]:
#Splitting the product categories into separate entities for filtering
import re
for i in range(len(df_fkrt)):
  a=df_fkrt.loc[i,"product_category_tree"].split(">>")
  tags=[re.sub("[^a-zA-Z]+"," ",i).strip() for i in a]
  df_fkrt.loc[i,"product_category_1"]=tags[0]
  try:
    df_fkrt.loc[i,"product_category_2"]=tags[1]
  except:
    df_fkrt.loc[i,"product_category_2"]=" "
  try:
    df_fkrt.loc[i,"product_category_3"]=tags[2:]
  except:
    df_fkrt.loc[i,"product_category_3"]=" "


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [8]:
#Dropping irrelevant columns
df_fkrt=df_fkrt.drop(["product_category_tree","uniq_id","crawl_timestamp","product_url","pid","is_FK_Advantage_product","overall_rating","product_rating"],axis=1)

In [9]:
#Splitting the product categories into separate entities for filtering
for i in range(len(df_amzn)):
  a=df_amzn.loc[i,"product_category_tree"].split(">>")
  tags=[re.sub("[^a-zA-Z]+"," ",i).strip() for i in a]
  df_amzn.loc[i,"product_category_1"]=tags[0]
  try:
    df_amzn.loc[i,"product_category_2"]=tags[1]
  except:
    df_amzn.loc[i,"product_category_2"]=" "
  try:
    df_amzn.loc[i,"product_category_3"]=",".join(tags[2:])
  except:
    df_amzn.loc[i,"product_category_3"]=" "


In [10]:
#Dropping irrelevant columns
df_amzn=df_amzn.drop(["product_category_tree","uniq_id","crawl_timestamp","product_url","pid","is_FK_Advantage_product","overall_rating","product_rating"],axis=1)

In [11]:
#sorting the datasets by product categories for more efficient calculations
df_fkrt=df_fkrt.sort_values(by=["product_category_1","product_category_2","product_category_3"])
df_amzn=df_amzn.sort_values(by=["product_category_1","product_category_2","product_category_3"])

<h1><b>Approach 1</b></h1>

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<b>Product matching via Word Embeddings and Cosine Similarity</b>

In [ ]:
#Using Sentence Transformer to get the word embeddings of the Product descriptions
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

In [ ]:
#Word embeddings
embeddings=model.encode(df_amzn["description"])

In [ ]:
#Result dataframe
res=pd.DataFrame(columns=["Product name in Flipkart","Retail Price in Flipkart","Discounted price in Flipkart",
                          "Product name in Amazon","Retail Price in Amazon","Discounted price in Amazon"])

In [ ]:
#Index of result rows
c=0

t1=time.perf_counter()
for i in range(len(df_fkrt)):
  try:
    #To store the brand name of the product from Flipkart
    tag1=df_fkrt.loc[i,"brand"]

    #To get all the products from Amazon that match the brand name
    amz1=df_amzn[df_amzn["brand"]==tag1]

    #To store the Product category of the product from Flipkart
    tag1=df_fkrt.loc[i,"product_category_1"]

    #To get all the products from Amazon that match the Product category 
    amz1=amz1[amz1["product_category_1"]==tag1]

    #To store the index of the matched products
    idx=[i for i in amz1.index]

    #To find the embeddings of the matched products
    embs=embeddings[idx]
    amz1.reset_index(drop=True,inplace=True)

    #To store the description of the product from Flipkart
    l_tag=df_fkrt.loc[i,"description"]
    embs=np.vstack((model.encode(l_tag),embs))

    #Calculating the cosine similarity of the product with Amazon products
    cs=cosine_similarity([embs[0]],embs[1:])
    cs=cs[0]
    k=np.argmax(cs)

    #To store the product name of the Flipkart item
    fk_name=df_fkrt.loc[i,"product_name"]

    #To store the retail of the Flipkart item
    fk_ret=df_fkrt.loc[i,"retail_price"]

    #To store the discounted price of the Flipkart item
    fk_dis=df_fkrt.loc[i,"discounted_price"]

    #To store the product name of the Amazon item
    amz_name=amz1.loc[k,"product_name"]

    #To store the retail price of the Amazon item
    amz_ret=amz1.loc[k,"retail_price"]

    #To store the discounted price of the Amazon item
    amz_dis=amz1.loc[k,"discounted_price"]

    
    res.loc[c]=[fk_name,fk_ret,fk_dis,amz_name,amz_ret,amz_dis]
    c+=1
  except:
    pass
t2=time.perf_counter()
print(np.round((t2-t1)/60,2)," minutes")


5.82  minutes


In [ ]:
res

In [ ]:
res.to_csv("Approach 1.csv")

<h1><b>Approach 2</b></h1>

<b>Product Matching via Image Similarity</b>

In [12]:
!pip install sewar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import re
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import cv2
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp


#Function to convert url of the images into OpenCV image files
def url_to_image(url):
  response = requests.get(url)
  image = Image.open(BytesIO(response.content)).convert("RGB")
  open_cv_image = np.array(image) 
  open_cv_image = open_cv_image[:, :, ::-1].copy()
  return open_cv_image

In [14]:
res=pd.DataFrame(columns=["Product name in Flipkart","Retail Price in Flipkart","Discounted price in Flipkart",
                          "Product name in Amazon","Retail Price in Amazon","Discounted price in Amazon"])

In [ ]:
c=0
t1=time.perf_counter()
for i in range(len(df_fkrt)):
  scores=[]
  try:

    #Storing the url of the Flipkart product image
    fk_img=re.findall("http:\S+.jpeg",df_fkrt.loc[i,"image"])

    #Storing the image converted from url
    fk_img=url_to_image(fk_img[0])

    #To store the brand name of the product from Flipkart
    tag1=df_fkrt.loc[i,"brand"]

    #To get all the products from Amazon that match the brand name
    amz1=df_amzn[df_amzn["brand"]==tag1]

    #To store the product category of the product from Flipkart
    tag1=df_fkrt.loc[i,"product_category_1"]

    #To get all the products from Amazon that match the product category
    amz1=amz1[amz1["product_category_1"]==tag1]

    #To store the second product category of the product from Flipkart    
    tag1=df_fkrt.loc[i,"product_category_2"]

    #To get all the products from Amazon that match the second product category
    amz1=amz1[amz1["product_category_2"]==tag1]

    amz1.reset_index(drop=True,inplace=True)

    for j in range(len(amz1)):

      #Storing the url of the Amazon product image
      img=re.findall("http:\S+.jpeg",amz1.loc[j,"image"])[0]

      #Storing the image converted from url
      image_from_url=url_to_image(img)

      #To store the height and width of the Amazon product image
      w,h=image_from_url.shape[:2]

      #Resizing the Amazon image to match the Flipkart image for finding the image distance
      fk_img=cv2.resize(fk_img,(h,w))

      #Image distance
      scores.append(mse(fk_img,image_from_url))
    
    #Minimum distance means best match
    idx=np.argmin(scores)

    fk_name=df_fkrt.loc[i,"product_name"]

    fk_ret=df_fkrt.loc[i,"retail_price"]

    fk_dis=df_fkrt.loc[i,"discounted_price"]

    amz_name=amz1.loc[idx,"product_name"]

    amz_ret=amz1.loc[idx,"retail_price"]

    amz_dis=amz1.loc[idx,"discounted_price"]

    res.loc[c]=[fk_name,fk_ret,fk_dis,amz_name,amz_ret,amz_dis]
    c+=1
  except:
    pass
t2=time.perf_counter() 
print(np.round((t2-t1)/60,2)," minutes")

In [ ]:
res

In [ ]:
res.to_csv('Approach 2.csv')